In [2]:
from torchvision import transforms
_tasks = transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
#The first transformation converts the raw data into tensor variables and 
#the second transformation performs normalization using the below operation:
#              x_normalized = x-mean / std
#The values 0.5 and 0.5 represent the mean and standard deviation for 3 channels: red, green, and blue.

In [4]:
from torchvision.datasets import MNIST
## Load MNIST Dataset and apply transformations
mnist = MNIST("data",download=True, train=True, transform=_tasks)

Processing...
Done!


In [27]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [28]:
#create training and validation split
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx,valid_idx = index_list[:split],index_list[split:]

In [29]:
#create sampler objects using SubsetRandomSampler
tr_sampler  = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [30]:
#create iterator objects for train and valid datasets
trainloader = DataLoader(mnist,batch_size=256, sampler = tr_sampler)
validloader = DataLoader(mnist,batch_size=256, sampler = val_sampler)

In [31]:
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784,128)
        self.output = nn.Linear(128,10)
    def forward(self,x):
        x = self.hidden(x)
        x = F.sigmoid(x)
        x = self.output(x)
        return x
model = Model()

In [32]:
from torch import optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,weight_decay = 1e-6, 
                     momentum = 0.9,nesterov = True)

In [33]:
for epoch in range(1,11): # running the model for 10 epochs
    train_loss,valid_loss =[], []
    ## training part
    model.train()
    for data,target in trainloader():
        optimizer.zero_grad()
        ##1. forward propagation
        output = model(data)
        
        ##2. loss calculation
        loss = loss_function(output,target)
        
        ##3. backward propagation
        loss.backward()
        
        ##4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss   = loss_function(ouput,target)
        valid_loss.append(loss.item())
    print("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))
        

TypeError: 'DataLoader' object is not callable